<a href="https://colab.research.google.com/github/mostafa-ja/Video_Dubber/blob/main/dubbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ffmpeg pysrt yt-dlp


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.6 MB/s eta 0:00:00
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=fcfd00a051b7d6e65a76451486f22001723ad2e19badf8e31275ec0081b6721c
  Stored in directory: /root/.cache/pip/wheels/56/30/c5/576bdd729f3bc062d62a551be7fefd6ed2f761901568171e4e
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=bb4017a92a057f384019040e876f7bed3bbb26ccf728805c0940612856c4f6cb
  Stored in directory: /root/.cache/pip/wheels/2d/b2/df/ea10959920533975b4a74a25a35e6d79655b63f3006611a99f
Successfully built ffmpeg pysrt


# Combine original and dubbed audio

## # upload dubbed audio

In [4]:
import shutil

# dubbed audio
source_path = "/content/drive/MyDrive/combined_audio.wav"
destination_path = "/content/combined_audio.wav"
shutil.copy(source_path, destination_path)

# translated srt
source_path = "/content/drive/My Drive/translated.srt"
destination_path = "/content/translated.srt"
shutil.copy(source_path, destination_path)


print("✅ Files copied from Google Drive.")

✅ File copied from Google Drive.


## extract original audio

In [3]:
import yt_dlp
import os

youtube_url = "https://www.youtube.com/watch?v=ISToBIkSNbM"
output_path="audio"

ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': output_path,
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }],
    'quiet': False
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=ISToBIkSNbM
[youtube] ISToBIkSNbM: Downloading webpage
[youtube] ISToBIkSNbM: Downloading tv client config
[youtube] ISToBIkSNbM: Downloading player 612f74a3-main
[youtube] ISToBIkSNbM: Downloading tv player API JSON
[youtube] ISToBIkSNbM: Downloading ios player API JSON
[youtube] ISToBIkSNbM: Downloading m3u8 information
[info] ISToBIkSNbM: Downloading 1 format(s): 251
[download] Destination: audio
[download] 100% of   16.53MiB in 00:00:00 at 30.46MiB/s  
[ExtractAudio] Destination: audio.wav
Deleting original file audio (pass -k to keep)


## mix audios

## mixing based on silence times

In [11]:
from pydub import AudioSegment, silence
import os

# === Load audio files ===
speech = AudioSegment.from_wav("combined_audio.wav")
bg = AudioSegment.from_file("audio.wav")  # Works for .mp3, .wav, etc.

# === Ensure background is long enough ===
if len(bg) < len(speech):
    bg *= ((len(speech) // len(bg)) + 1)
#bg = bg[:len(speech)]

# === Detect non-silent parts of speech ===
nonsilent_ranges = silence.detect_nonsilent(
    speech,
    min_silence_len=200,
    silence_thresh=speech.dBFS - 16
)

# === Volume settings ===
speech_gain = 5
bg_quiet_gain = -20  # Lower volume during speech
fade_duration = 1000  # in ms, for fade in/out

# === Start with background as base ===
output = bg

# === Process each speech segment ===
for start, end in nonsilent_ranges:
    # Apply fades to background and mix with speech
    speech_chunk = speech[start:end] + speech_gain
    bg_chunk = bg[start:end] + bg_quiet_gain

    # Add fade-in and fade-out for smoother transitions
    #speech_chunk = speech_chunk.fade_in(fade_duration).fade_out(fade_duration)
    bg_chunk = bg_chunk.fade_in(fade_duration).fade_out(fade_duration)

    mixed = bg_chunk.overlay(speech_chunk)

    # Replace that portion in output
    output = output[:start] + mixed + output[end:]

# === Export as WAV ===
output.export("final_mixed_audio.wav", format="wav")
print("✅ Saved as 'final_mixed_audio.wav'")

# === Export as MP3 (compressed) ===
output.export("final_mixed_audio.mp3", format="mp3", bitrate="192k")
print("🎧 Saved as 'final_mixed_audio.mp3'")


✅ Saved as 'final_mixed_audio.wav'
🎧 Saved as 'final_mixed_audio.mp3'


## ## mixing based on subtitle times

In [11]:
from pydub import AudioSegment, silence
import pysrt
import os

# === Load audio files ===
speech = AudioSegment.from_wav("combined_audio.wav")
bg = AudioSegment.from_file("audio.wav")  # Works for .mp3, .wav, etc.

# === Load subtitles ===
subs = pysrt.open('translated.srt', encoding='utf-8')

# === Ensure background is long enough ===
if len(bg) < len(speech):
    bg *= ((len(speech) // len(bg)) + 1)
#bg = bg[:len(speech)]

# === Volume settings ===
speech_gain = 5
bg_quiet_gain = -20  # Lower volume during speech
fade_duration = 500  # in ms, for fade in/out

# === Start with background as base ===
output = bg

# === Process each speech segment ===
for i, sub in enumerate(subs):

    # Time window in ms
    start = (sub.start.hours * 3600 + sub.start.minutes * 60 + sub.start.seconds) * 1000 + sub.start.milliseconds
    end = (sub.end.hours * 3600 + sub.end.minutes * 60 + sub.end.seconds) * 1000 + sub.end.milliseconds
    time_window = end - start

    # Apply fades to background and mix with speech
    speech_chunk = speech[start:end] + speech_gain
    bg_chunk = bg[start:end] + bg_quiet_gain

    # Add fade-in and fade-out for smoother transitions
    #speech_chunk = speech_chunk.fade_in(fade_duration).fade_out(fade_duration)
    bg_chunk = bg_chunk.fade_in(fade_duration).fade_out(fade_duration)

    mixed = bg_chunk.overlay(speech_chunk)

    # Replace that portion in output
    output = output[:start] + mixed + output[end:]

# === Export as WAV ===
output.export("final_mixed_audio.wav", format="wav")
print("✅ Saved as 'final_mixed_audio.wav'")

# === Export as MP3 (compressed) ===
output.export("final_mixed_audio.mp3", format="mp3", bitrate="192k")
print("🎧 Saved as 'final_mixed_audio.mp3'")


✅ Saved as 'final_mixed_audio.wav'
🎧 Saved as 'final_mixed_audio.mp3'


# Download video

In [7]:
# checking qualities
!yt-dlp -F "https://www.youtube.com/watch?v=ISToBIkSNbM"

[youtube] Extracting URL: https://www.youtube.com/watch?v=ISToBIkSNbM
[youtube] ISToBIkSNbM: Downloading webpage
[youtube] ISToBIkSNbM: Downloading tv client config
[youtube] ISToBIkSNbM: Downloading tv player API JSON
[youtube] ISToBIkSNbM: Downloading ios player API JSON
[youtube] ISToBIkSNbM: Downloading m3u8 information
[info] Available formats for ISToBIkSNbM:
ID      EXT   RESOLUTION FPS CH │   FILESIZE   TBR PROTO │ VCODEC          VBR ACODEC      ABR ASR MORE INFO
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
sb3     mhtml 48x27        0    │                  mhtml │ images                                  storyboard
sb2     mhtml 80x45        0    │                  mhtml │ images                                  storyboard
sb1     mhtml 160x90       0    │                  mhtml │ images                                  storyboard
sb0     mhtml 320x180      0    │                  mhtml │ images 

## testing

In [12]:
# downloading low quality for test

ydl_opts = {
    'format': '394+140',  # 1080p video + good m4a audio
    'outtmpl': 'test_video.mp4',
    'merge_output_format': 'mp4'
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(["https://www.youtube.com/watch?v=ISToBIkSNbM"])


[youtube] Extracting URL: https://www.youtube.com/watch?v=ISToBIkSNbM
[youtube] ISToBIkSNbM: Downloading webpage
[youtube] ISToBIkSNbM: Downloading tv client config
[youtube] ISToBIkSNbM: Downloading tv player API JSON
[youtube] ISToBIkSNbM: Downloading ios player API JSON
[youtube] ISToBIkSNbM: Downloading m3u8 information
[info] ISToBIkSNbM: Downloading 1 format(s): 394+140
[download] test_video.mp4 has already been downloaded


In [13]:
import subprocess

# Replace audio using ffmpeg
subprocess.call([
    'ffmpeg', '-y',
    '-i', 'test_video.mp4',
    '-i', 'final_mixed_audio.wav',
    '-c:v', 'copy',           # Copy video without re-encoding
    '-map', '0:v:0',          # Use video from first input
    '-map', '1:a:0',          # Use audio from second input
    '-shortest',              # Trim to shorter of video or audio
    'test_video_with_new_audio.mp4'
])

0

## mixing high quality video and dubbed audio

In [14]:
ydl_opts = {
    'format': '299+140',  # 1080p video + good m4a audio
    'outtmpl': 'video_1080p.mp4',
    'merge_output_format': 'mp4'
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download(["https://www.youtube.com/watch?v=ISToBIkSNbM"])


[youtube] Extracting URL: https://www.youtube.com/watch?v=ISToBIkSNbM
[youtube] ISToBIkSNbM: Downloading webpage
[youtube] ISToBIkSNbM: Downloading tv client config
[youtube] ISToBIkSNbM: Downloading tv player API JSON
[youtube] ISToBIkSNbM: Downloading ios player API JSON
[youtube] ISToBIkSNbM: Downloading m3u8 information
[info] ISToBIkSNbM: Downloading 1 format(s): 299+140
[download] Destination: video_1080p.f299.mp4
[download] 100% of  589.03MiB in 00:00:09 at 59.65MiB/s  
[download] Destination: video_1080p.f140.m4a
[download] 100% of   16.63MiB in 00:00:00 at 50.73MiB/s  
[Merger] Merging formats into "video_1080p.mp4"
Deleting original file video_1080p.f299.mp4 (pass -k to keep)
Deleting original file video_1080p.f140.m4a (pass -k to keep)


In [ ]:
from yt_dlp import YoutubeDL

youtube_url = "https://www.youtube.com/watch?v=ISToBIkSNbM"
video_filename = "original_video.mp4"

ydl_opts = {
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
    'outtmpl': video_filename,
    'merge_output_format': 'mp4'
}

with YoutubeDL(ydl_opts) as ydl:
    ydl.download([youtube_url])


In [15]:
import subprocess

# Replace audio using ffmpeg
subprocess.call([
    'ffmpeg', '-y',
    '-i', 'video_1080p.mp4',
    '-i', 'final_mixed_audio.wav',
    '-c:v', 'copy',           # Copy video without re-encoding
    '-map', '0:v:0',          # Use video from first input
    '-map', '1:a:0',          # Use audio from second input
    '-shortest',              # Trim to shorter of video or audio
    'dubbed_video.mp4'
])


0

## upload result to google drive

In [17]:
import shutil

source_path = "/content/dubbed_video.mp4"
destination_path = "/content/drive/MyDrive/dubbed_video.mp4"
shutil.copy(source_path, destination_path)



print("✅ File copied from Google Drive.")

✅ File copied from Google Drive.
